In [1]:
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
import numpy as np
from sklearn import preprocessing, model_selection
from sklearn.ensemble import RandomForestRegressor, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree, ensemble, linear_model
from sklearn import metrics
from sklearn import cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
frogs = pd.read_csv("data/Frogs_MFCCs.csv").drop(['Genus', 'Species', 'RecordID'],axis=1)
frogs['Family'] = frogs['Family'].map({'Dendrobatidae': 1}).fillna(0)
frogs.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,0.0
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,0.0
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,0.0
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,0.0
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,0.0


In [3]:
X = frogs.drop('Family', axis=1)
y = frogs['Family']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, test_size=0.2, random_state=31)


In [4]:
rf = ensemble.RandomForestClassifier(10, random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
print(f"rf results f1: {metrics.f1_score(y_test, preds)}")

rf results f1: 0.9726775956284154


In [5]:
stack = ensemble.StackingClassifier(
    [('rf', ensemble.RandomForestClassifier(10,random_state=31)),
    ('knn', KNeighborsClassifier(11)),
    ('nb', GaussianNB())
    ]
)
stack.fit(X_train, y_train)
preds = stack.predict(X_test)
print(f"stacking results f1: {metrics.f1_score(y_test, preds)}")

stacking results f1: 0.989247311827957


In [6]:
from joblib import dump, load
dump(stack, 'stack.joblib') 
stack_ = load('stack.joblib') 
stack_

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=31)),
                               ('knn', KNeighborsClassifier(n_neighbors=11)),
                               ('nb', GaussianNB())])